<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2

# Для подавления сообщений об ошибках при обработке pandas ответов psycopg2
import warnings
warnings.filterwarnings('ignore')

In [2]:
''' Необходимые для подключения к БД данные содержатся в отдельном файле
    "creds.py" для безопасности. Параметры импортируются в словарь CONN_ATTR,
    который далее используется в функции "make_request()" при запросах к базе.
'''
import creds as c_

CONN_ATTR = {
    'user': c_.USER,
    'password': c_.PASSWORD,
    'host': c_.HOST,
    'port': c_.PORT,
    'dbname': c_.DBNAME
    }

def make_request(query_: str) -> object:
    ''' Делает запросы к БД с определёнными аттрибутами
    Arguments:
        query_ [str] -- Строка SQL-запроса (можно многострочник)
    Returns:
        [object] -- Ответ на запрос
    '''
    with psycopg2.connect(**CONN_ATTR) as conn:
        return pd.read_sql_query(query_, conn)


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [3]:
# текст запроса
query_3_1 = f'''
SELECT COUNT(*)
FROM public.vacancies
'''

In [4]:
# результат запроса
display(make_request(query_3_1))

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''
SELECT COUNT(*)
FROM public.employers
'''

In [7]:
# результат запроса
display(make_request(query_3_2))

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''
SELECT COUNT(*)
FROM public.areas
'''

In [9]:
# результат запроса
display(make_request(query_3_3))

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''
SELECT COUNT(*)
FROM public.industries
'''

In [11]:
# результат запроса
display(make_request(query_3_4))

,count
0,294


***

## Выводы по предварительному анализу данных ##

Пока ничего особенного. Получены размеры таблиц, от которых уже можно далее вести оценочные расчёты. Примерно две вакансии на работодателя ещё ни о чём не говорят. Будем проводить дополнительные исследования ниже.

Интересно было бы посмотреть на распределение вакансий по отраслям и по географии, но, к сожалению, в таблицах нет полезных дополнительных идентификаторов регионов/стран и обобщённых видов деятельности компаний.

----


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''
SELECT
    a.name AS area,
    COUNT(*) AS cnt
FROM public.vacancies AS v
JOIN public.areas AS a ON
    v.area_id = a.id
GROUP BY a.id
ORDER BY cnt DESC
-- Выставляем лимит больше, чем в задании (5). Интересно посмотреть.
LIMIT 20
'''

In [13]:
# результат запроса
display(make_request(query_4_1))

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    salary_from IS NOT NULL
    OR
    salary_to IS NOT NULL
'''

In [15]:
# результат запроса
display(make_request(query_4_2))

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''
SELECT
    -- Применяется преобразование к целочисленному типу.
    -- Для надёжности предварительно используется round() для округления,
    -- хотя в данном случае можно и без него.
    ROUND(AVG(salary_from))::integer AS avg_min,
    ROUND(AVG(salary_to))::integer AS avg_max
FROM public.vacancies
'''

In [17]:
# результат запроса
display(make_request(query_4_3))

,avg_min,avg_max
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''
SELECT
    schedule,
    employment,
    COUNT(*) AS cnt
FROM public.vacancies
GROUP BY schedule, employment
ORDER BY cnt DESC
-- Для строгого ответа на вопрос задания можно раскомментить ниже
-- OFFSET 1
-- LIMIT 1
'''

In [19]:
# результат запроса
display(make_request(query_4_4))

,schedule,employment,cnt
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''
SELECT
    experience AS "Требуемый опыт работы",
    -- Для оптимизации сортировки добавить счётчик
    COUNT(*) AS cnt
FROM public.vacancies
GROUP BY experience
ORDER BY cnt
'''

In [21]:
# результат запроса
display(make_request(query_4_5))

,Требуемый опыт работы,cnt
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы по детальному анализу вакансий ##

В лидирующую пятёрку лидеров-регионов ожидаемо вошли столичные города РФ, РБ и РК. Интересно то, что &laquo;столица&raquo; Сибири (Новосибирск) обогнала &laquo;столицу&raquo; Урала (Екатеринбург). Это может означать, что в СФО началось опережающее развитие, либо просто на Урале больше крупных городов, по которым &laquo;распыляются&raquo; вакансии федерального округа. Жаль, что для уточнения в БД нет дополнительных региональных идентификаторов.

Радует, что почти 19% вакансий (как посчитано в дополнительных исследованиях ниже)&nbsp;&mdash; удалёнка.

Примерно для половины вакансий заданы пределы по зарплате (то есть уже определён бюджет), остальные гибко &laquo;обсуждаемы&raquo; (или скрыты из соображений коммерческой тайны).

Помимо всего, рынок вакансий выглядит достаточно динамичным (почти 68% вакансий не требуют высокой квалификации&nbsp;&mdash; только от года до трёх или вообще без опыта).

----


# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''
(
    SELECT
        e.name,
        COUNT(*) AS vacancies_number
    FROM public.vacancies AS v
    JOIN public.employers AS e ON
        v.employer_id = e.id
    GROUP BY e.name
    ORDER BY vacancies_number DESC
    LIMIT 1
)
UNION ALL
(
    SELECT
        ee.name,
        COUNT(*) AS vn
    FROM public.vacancies AS vv
    JOIN public.employers AS ee ON
        ee.id = vv.employer_id
    GROUP BY ee.name
    ORDER BY vn DESC
    OFFSET 4
    LIMIT 1
)
'''

In [23]:
# результат запроса
display(make_request(query_5_1))

,name,vacancies_number
0,Яндекс,1933
1,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''
SELECT
    a.name,
    COUNT(DISTINCT e.id) AS employers_cnt,
    COUNT(DISTINCT v.id) AS vacancies_cnt
FROM public.areas AS a
LEFT JOIN public.employers AS e ON
    a.id = e.area
LEFT JOIN public.vacancies AS v ON
    a.id = v.area_id
GROUP BY a.id
ORDER BY vacancies_cnt, employers_cnt DESC
-- Для строгого ответа на вопрос задания можно раскомментить ниже
-- LIMIT 1
'''

In [25]:
# результат запроса
display(make_request(query_5_2))

,name,employers_cnt,vacancies_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [57]:
# текст запроса
query_5_3 = f'''
SELECT
    e.name,
    COUNT(DISTINCT v.area_id) AS area_cnt
FROM public.employers AS e
LEFT JOIN public.vacancies AS v ON
    e.id = v.employer_id
GROUP BY e.id
ORDER BY area_cnt DESC
LIMIT 20
'''

In [58]:
# результат запроса
display(make_request(query_5_3))

,name,area_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,КРОН,48
9,Почта России,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''
SELECT
    COUNT(DISTINCT e.id)
FROM public.employers AS e
LEFT JOIN public.employers_industries AS ei ON
    e.id = ei.employer_id
WHERE ei.industry_id IS NULL
'''

In [29]:
# результат запроса
display(make_request(query_5_4))

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''
SELECT
    e.name,
    COUNT(*)
FROM public.employers_industries AS ei
JOIN public.employers AS e ON
    e.id = ei.employer_id
GROUP BY e.id
HAVING COUNT(*) = 4
ORDER BY e.name
OFFSET 2
LIMIT 1
'''

In [31]:
# результат запроса
display(make_request(query_5_5))

,name,count
0,2ГИС,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''
SELECT
    COUNT(e.id)
FROM public.employers AS e
JOIN public.employers_industries AS ei ON
    e.id = ei.employer_id
JOIN public.industries AS i ON
    i.id = ei.industry_id
    AND
    i.name = 'Разработка программного обеспечения'
'''

In [33]:
# результат запроса
display(make_request(query_5_6))

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [23]:
# код для получения списка городов-милионников

# REQUIRED pip install requests
import requests
# REQUIRED pip install beautifulsoup4
from bs4 import BeautifulSoup as bs

wiki_url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8'

response = requests.get(wiki_url)
page = bs(response.text, 'html.parser')
megas_list = []
tag_list = page.find('table', class_='standard sortable').find_all('a')
for tag_ in tag_list:
    str_ = tag_.text
    if str_[0].isupper():
        megas_list.append(str_)
megas_str = "('" + "', '".join(megas_list) + "')"
print(megas_str)

('Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград')


In [4]:
# текст запроса
query_5_7 = f'''
(
    SELECT
        a.name,
        COUNT(v.id) AS cnt
    FROM public.vacancies AS v
    JOIN public.employers AS e ON
        e.id = v.employer_id
        AND
        e.name = 'Яндекс'
    JOIN public.areas AS a ON
        a.id = v.area_id
        AND
        a.name IN {megas_str}
    GROUP BY a.name
)

UNION ALL

(
    SELECT
        'Total',
        COUNT(*)
    FROM public.vacancies AS vv
    JOIN public.employers AS ee ON
        ee.id = vv.employer_id
        AND
        ee.name = 'Яндекс'
    JOIN public.areas AS aa ON
        aa.id = vv.area_id
        AND
        aa.name IN {megas_str}
)
ORDER BY cnt
'''

In [5]:
# результат запроса
display(make_request(query_5_7))

,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

## Выводы по анализу работодателей ##

Всё-таки в базе есть не только населённые пункты, но и регионы, которые указывают работодатели. Однако работодатели не указывают вакансии в них, а только в городах.

Специалисту по Data Science полезно отслеживать появление вакансий в компаниях, находящихся в верхних строках таблицы по присутствия в регионах (5.3). Они имеют развитую информационную инфраструктуру, следовательно, большие объёмы и потоки разных данных, которые надо обрабатывать. Если даже не все из них положительно относятся к удалёнке, есть шансы, что их филиалы появятся в городе проживания кандидата.

Около 15% работодателей занимаются разработкой ПО, что тоже представляет интерес. Трудно сказать, много это или мало для экономики в целом, зависит от релевантности выборки из баз данных hh.ru.

Полученные данные по &laquo;Яндексу&raquo; в реальности мало что дают, поскольку большая часть вакансий всё равно с возможностью удалённой работы без жёсткой привязки к городу.

----


# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [6]:
# текст запроса
query_6_1 = f'''
SELECT
    COUNT(name)
FROM public.vacancies
WHERE
    LOWER(name) LIKE '%data%'
    OR
    LOWER(name) LIKE '%данн%'
'''

In [7]:
# результат запроса
display(make_request(query_6_1))

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [30]:
# текст запроса
query_6_2 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    (
        LOWER(name) LIKE '%junior%'
        OR
        experience = 'Нет опыта'
        OR
        employment = 'Стажировка'
    )
'''

In [31]:
# результат запроса
display(make_request(query_6_2))

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [34]:
# текст запроса
query_6_3 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    (
        LOWER(key_skills) LIKE '%sql%'
        OR
        LOWER(key_skills) LIKE '%postgres%'
    )
'''

In [35]:
# результат запроса
display(make_request(query_6_3))

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [36]:
# текст запроса
query_6_4 = f'''
SELECT
    COUNT(*)
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    LOWER(key_skills) LIKE '%python%'
'''

In [37]:
# результат запроса
display(make_request(query_6_4))

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [47]:
# текст запроса
query_6_5 = f'''
SELECT
    -- Для вычисления в строке из "key_skills" ключевых навыков, разделённых
    -- табуляторами, надо из длины всей строки вычесть её же длину без табов
    -- плюс один
    ROUND(AVG(LENGTH(key_skills) - LENGTH(REPLACE(key_skills, CHR(9), '')) + 1), 2) AS round_avg
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND key_skills IS NOT NULL
'''

In [43]:
# результат запроса
display(make_request(query_6_5))

,round_avg
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [51]:
# текст запроса
query_6_6 = f'''
SELECT
    experience AS "Опыт",
    AVG((COALESCE(salary_from, salary_to) + COALESCE(salary_to, salary_from)) / 2)::integer AS "Средняя зарплата в DS"
FROM public.vacancies
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    (
        salary_from IS NOT NULL
        OR
        salary_to IS NOT NULL
    )
GROUP BY experience
'''

In [52]:
# результат запроса
display(make_request(query_6_6))

,Опыт,Средняя зарплата в DS
0,Нет опыта,74643
1,От 1 года до 3 лет,139675
2,От 3 до 6 лет,243115


***

## Выводы по предметному анализу ##

По таблице (6.6) на самом деле вакансии для опыта более 6 лет могут быть, но вилка зарплат там не указывается. Видимо настолько высокая или гибкая, что все детали решаются при собеседовании. И для совсем начинающих специалистов средняя зарплата выглядит вполне оптимистично.

Но не стоит расслабляться. Конечно, &laquo;чистый&raquo; Python требуется чаще, чем знания по базам данных, однако таблица (6.5) намекает на то, что дополнительные входные требования по hardskills к DS-специалисту предъявляются высокие. Не просто Python + SQL + PostgreSQL.

----


# Дополнительные исследования #

## Какая доля удалёнки по типам занятости? ##
#### (Дополнение к выводам раздела 4 по детальному анализу вакансий) ####

In [77]:
''' Какой процент вакансий по каждому типу занятости составляют вакансии
    с возможностью удалённой работы?
'''
query_add02 = f'''
SELECT
    v.employment,
    COUNT(vr.id) AS remote_vacancies,
    COUNT(v.id) AS total_vacancies,
    COUNT(vr.id)::real / COUNT(v.id)::real * 100 AS remote_percent
FROM public.vacancies AS v
LEFT JOIN public.vacancies AS vr ON
    v.id = vr.id
    AND
    vr.schedule = 'Удаленная работа'
GROUP BY v.employment

UNION ALL

SELECT
    'Всего:',
    COUNT(vr.id),
    COUNT(v.id),
    COUNT(vr.id)::real / COUNT(v.id)::real * 100
FROM public.vacancies AS v
LEFT JOIN public.vacancies AS vr ON
    v.id = vr.id
    AND
    vr.schedule = 'Удаленная работа'

ORDER BY remote_vacancies
'''

In [78]:
display(make_request(query_add02))

,employment,remote_vacancies,total_vacancies,remote_percent
0,Стажировка,64,761,8.409987
1,Проектная работа,133,295,45.084745
2,Частичная занятость,1312,2072,63.320464
3,Полная занятость,7802,46069,16.935466
4,Всего:,9311,49197,18.925951


## А что в Улан-Удэ? ##
#### (Какие возможности в небольших отдалённых городах?) ####

In [11]:
''' Чтобы не дёргать лишний раз таблицу 'areas' при запросах,
    выясним id города Улан-Удэ
'''
query_uu01 = '''
SELECT id
FROM public.areas
WHERE name = 'Улан-Удэ'
'''
uu_id = make_request(query_uu01)['id'].iloc[0]
print('Идентификатор Улан-Удэ:', uu_id)

Идентификатор Улан-Удэ: 20


In [18]:
''' Сколько всего вакансий в городе? '''
query_uu02 = f'''
SELECT
    COUNT(*) AS "Всего вакансий в Улан-Удэ"
FROM public.vacancies
WHERE area_id = 20
'''
display(make_request(query_uu02))

,Всего вакансий в Улан-Удэ
0,84


In [17]:
''' Есть вакансии для DS-спецов? '''
query_uu03 = f'''
SELECT COUNT(*)
FROM public.vacancies AS v
WHERE
    (
        LOWER(name) LIKE '%data scientist%'
        OR
        LOWER(name) LIKE '%data science%'
        OR
        LOWER(name) LIKE '%исследователь данных%'
        OR
        (
            name NOT LIKE '%HTML%'
            AND
            name LIKE '%ML%'
        )
        OR
        LOWER(name) LIKE '%machine learning%'
        OR
        LOWER(name) LIKE '%машинн%обучен%'
    )
    AND
    (
        salary_from IS NOT NULL
        OR
        salary_to IS NOT NULL
    )
    AND
    v.area_id = 20
'''
display(make_request(query_uu03))

,count
0,0


In [22]:
''' Есть ли возможности для разработчика? '''
query_uu04 = f'''
SELECT
    v.name AS "Вакансия",
    v.experience AS "Опыт",
    v.schedule AS "График",
    v.employment AS "Тип",
    e.name AS "Компания"
FROM public.vacancies AS v
JOIN public.employers AS e ON
    v.employer_id = e.id
    AND
    v.area_id = 20
JOIN public.employers_industries AS ei ON
    e.id = ei.employer_id
JOIN public.industries AS i ON
    i.id = ei.industry_id
    AND
    i.name = 'Разработка программного обеспечения'
'''
display(make_request(query_uu04))

,Вакансия,Опыт,График,Тип,Компания
0,Специалист внедрения/Специалист технической по...,От 1 года до 3 лет,Удаленная работа,Полная занятость,ИнтеллектДиалог
1,Старший инженер облачной инфраструктуры,От 3 до 6 лет,Полный день,Полная занятость,МегаФон


In [19]:
''' Что вообще может предложить Яндекс помимо техподдержки? '''
query_uu05 = f'''
SELECT
    v.name AS "Вакансия",
    v.experience AS "Опыт",
    v.employment AS "Тип",
    v.schedule AS "График",
    a.name
FROM public.vacancies AS v
JOIN public.employers AS e ON
    e.id = v.employer_id
    AND
    e.name = 'Яндекс'
    AND
    v.name NOT LIKE '%поддержк%'
JOIN public.areas AS a ON
    a.id = v.area_id
    -- Включим в поиск соседние города
    AND
    a.name IN ('Улан-Удэ', 'Иркутск', 'Чита')
ORDER BY a.name
'''
display(make_request(query_uu05))

,Вакансия,Опыт,Тип,График,name
0,Младший специалист ручного тестирования,Нет опыта,Частичная занятость,Удаленная работа,Иркутск
1,Специалист по общению с клиентами в Яндекс Бизнес,Нет опыта,Полная занятость,Удаленная работа,Иркутск
2,Эксперт по ответам на вопросы о Яндекс Бизнесе,Нет опыта,Полная занятость,Удаленная работа,Иркутск
3,Технический писатель (дистанционный автор),От 1 года до 3 лет,Частичная занятость,Удаленная работа,Иркутск
4,Тимлид клиентского сервиса,От 1 года до 3 лет,Полная занятость,Удаленная работа,Иркутск
5,Разработчик (поисковая выдача),Нет опыта,Полная занятость,Удаленная работа,Иркутск
6,Младший специалист ручного тестирования,Нет опыта,Частичная занятость,Удаленная работа,Улан-Удэ
7,Разработчик (поисковая выдача),Нет опыта,Полная занятость,Удаленная работа,Улан-Удэ
8,Технический писатель (дистанционный автор),От 1 года до 3 лет,Частичная занятость,Удаленная работа,Улан-Удэ
9,Технический писатель (дистанционный автор),От 1 года до 3 лет,Частичная занятость,Удаленная работа,Чита


# Общий вывод по проекту

В части удалённой работы отрадно, что она составляет очень большую долю в проектной работе и частичной занятости. Оптимальный вариант для начинающих специалистов из небольших городов-&laquo;недомиллионников&raquo; в экономически депрессивном регионе, которые при этом плотно учатся и/или подрабатывают.

А вот устроиться на стажировку уже тяжело. Офис должен быть в пределах ежедневной доступности.

Локальных вакансий даже в разработке ПО слишком мало, а для специалиста по DS нет совсем. Поэтому, помимо вышеуказанных возможностей с part-time удалёнкой, и если отбросить варианты с релокацией, то оставшийся путь&nbsp;&mdash; полная удалёнка (full-remote). Но это тяжёлый вариант, поскольку работодатели до сих пор не любят брать новичка без обязательного надзора за ним в офисе хотя бы на испытательном сроке.

Можно отслеживать крупные компаний, наподобие Яндекса, где могут мелькать вакансии, интересные и для начинающего DS-ника/питониста (вроде разработчика на поисковой выдаче).

В целом ещё было бы интересно проанализировать в динамике появление и закрытие вакансий, изменения их соотношений по разным параметрам. Для этого было бы полезно добавить в выборку данных различные временные метки.

Возникает идея пет-проекта для автоматизации поисков. Попытаться использовать API с бесплатным доступом на hh.ru хотя бы для зарегистрированных кандидатов. Как альтернатива, можно попробовать создать собственный вэб-скрапер сайта.

----
